In [19]:
import pandas as pd
import numpy as np
import pydot

In [20]:
data = pd.read_csv('/home/akash/Downloads/tennis.csv')
labels = data['PlayTennis']
classes = np.unique(labels)
print(data)

     Outlook Temperature Humidity  Windy PlayTennis
0      Sunny         Hot     High  False         No
1      Sunny         Hot     High   True         No
2   Overcast         Hot     High  False        Yes
3      Rainy        Mild     High  False        Yes
4      Rainy        Cool   Normal  False        Yes
5      Rainy        Cool   Normal   True         No
6   Overcast        Cool   Normal   True        Yes
7      Sunny        Mild     High  False         No
8      Sunny        Cool   Normal  False        Yes
9      Rainy        Mild   Normal  False        Yes
10     Sunny        Mild   Normal   True        Yes
11  Overcast        Mild     High   True        Yes
12  Overcast         Hot   Normal  False        Yes
13     Rainy        Mild     High   True         No


In [21]:
def entropy(S):
    #print(S)
    labels = S['PlayTennis']
    a = sorted(labels.value_counts().to_dict())
    #print(a)
    classes = np.unique(labels) 
    #print(labels)

    c = labels.value_counts()
    no_of_classes = len(c)
    
    en = 0
    
    p = []
    
    for i in range(no_of_classes):
        p.append(float(c[i]/len(labels)))
    #print(p)    
    
    for i in range(len(p)):
        en += -1 * p[i] * np.log2(p[i])
          
    return en
    

In [22]:
en=entropy(data)
print('Entropy', en)

Entropy 0.9402859586706309


In [23]:
d= data.loc[data['Outlook']=='Sunny']
print(entropy(d))

0.9709505944546686


In [24]:
def gain(S,A):
    n=[]
    e=[]
    s=[]
    count=[]
    a=np.unique(S[A])
    no_of_values=len(a)
    c= S[A].value_counts().to_dict()
    print(c)
    for i in a:
        count.append(c[i])
    
    print(S[A].value_counts())   
    
    #print(a)
    #print(no_of_values)
    #print(len(S))
     
    
    for i in range(no_of_values):
        n.append((float(count[i]))/len(S))
        
        #print(count[i])
        d = S.loc[S[A]==a[i]]
        
        #print(d)
        e.append(entropy(d))
        #s.append(n[i]*e[i])
    #print(n)
    #print("\n")
    #print(e)  
    e=np.array(e)
    n=np.array(n)
    #print(e)
    s=n*e
    su=np.sum(s)
    #print("Unique values: \n",a)
    #print(count)
    #print(n)
    #print(e)
    #su=sum(s)
    g= entropy(S) - su
    
    return g

In [25]:
print(gain(data,'Outlook'))

{'Sunny': 5, 'Rainy': 5, 'Overcast': 4}
Sunny       5
Rainy       5
Overcast    4
Name: Outlook, dtype: int64
0.2467498197744391


In [26]:
print(gain(data, 'Windy'))

{False: 8, True: 6}
False    8
True     6
Name: Windy, dtype: int64
0.04812703040826927


In [27]:
print(gain(data, 'Humidity'))

{'Normal': 7, 'High': 7}
Normal    7
High      7
Name: Humidity, dtype: int64
0.15183550136234136


In [28]:
print(gain(data, 'Temperature'))

{'Mild': 6, 'Cool': 4, 'Hot': 4}
Mild    6
Cool    4
Hot     4
Name: Temperature, dtype: int64
0.029222565658954647


In [29]:
S_sunny=data.loc[data['Outlook']=='Sunny']
print(gain(S_sunny,'Windy'))

{False: 3, True: 2}
False    3
True     2
Name: Windy, dtype: int64
0.01997309402197489


In [30]:
S_temp=data.loc[data['Outlook']=='Sunny']
print(gain(S_temp,'Temperature'))

{'Hot': 2, 'Mild': 2, 'Cool': 1}
Hot     2
Mild    2
Cool    1
Name: Temperature, dtype: int64
0.5709505944546686


In [31]:
S_temp=data.loc[data['Outlook']=='Rainy']
print(gain(S_temp,'Temperature'))

{'Mild': 3, 'Cool': 2}
Mild    3
Cool    2
Name: Temperature, dtype: int64
0.01997309402197489


In [32]:
S_temp=data.loc[data['Outlook']=='Rainy']
print(gain(S_temp,'Windy'))

{False: 3, True: 2}
False    3
True     2
Name: Windy, dtype: int64
0.9709505944546686


In [38]:
#en = entropy(data)
#x = list(data.columns)
#x.pop()
#print(x)
#ent ={}

#for i in x:
#    ent[i] = gain(data,i)

#v = list(ent.values())
#k = list(ent.keys())
#max_gain_attribute = k[v.index(max(v))]
    
#print("\n\nAttribute with the max gain: ",max_gain_attribute)   


def tree(data):
    
    #attrib_vals = []
    #for i in range(len(x)):
    #    attrib_vals.append(np.unique(data[x[i]]))
        
    #print(attrib_vals)    
    #print(x)
    
    feat = data.iloc[:,:-1].columns
    dt = {}
    g = {}
    
    for i in feat:
        g[i] = gain(data,i)
        
    print(g)    
    
    max_gain_attrib = max(g, key=g.get)
    print(max_gain_attrib)
    
    attrib_vals = data.loc[:,max_gain_attrib]
    print(attrib_vals)
    
    dt[max_gain_attrib] = {}
    for i in np.unique(attrib_vals):
        
        if len(np.unique(data.loc[data[max_gain_attrib]==i,:].iloc[:,-1])) > 1:
            
            dt[max_gain_attrib][i] = tree(data.loc[data[max_gain_attrib]==i,:].drop(max_gain_attrib,axis=1))
            
        else:
            print("Leaf:", max_gain_attrib, " at:", i)
            dt[max_gain_attrib][i] =  str(np.unique(data.loc[data[max_gain_attrib]==i,:].iloc[:,-1])[0])
    return dt

In [39]:
decision_tree = tree(data)

{'Sunny': 5, 'Rainy': 5, 'Overcast': 4}
Sunny       5
Rainy       5
Overcast    4
Name: Outlook, dtype: int64
{'Mild': 6, 'Cool': 4, 'Hot': 4}
Mild    6
Cool    4
Hot     4
Name: Temperature, dtype: int64
{'Normal': 7, 'High': 7}
Normal    7
High      7
Name: Humidity, dtype: int64
{False: 8, True: 6}
False    8
True     6
Name: Windy, dtype: int64
{'Outlook': 0.2467498197744391, 'Temperature': 0.029222565658954647, 'Humidity': 0.15183550136234136, 'Windy': 0.04812703040826927}
Outlook
0        Sunny
1        Sunny
2     Overcast
3        Rainy
4        Rainy
5        Rainy
6     Overcast
7        Sunny
8        Sunny
9        Rainy
10       Sunny
11    Overcast
12    Overcast
13       Rainy
Name: Outlook, dtype: object
Leaf: Outlook  at: Overcast
{'Mild': 3, 'Cool': 2}
Mild    3
Cool    2
Name: Temperature, dtype: int64
{'Normal': 3, 'High': 2}
Normal    3
High      2
Name: Humidity, dtype: int64
{False: 3, True: 2}
False    3
True     2
Name: Windy, dtype: int64
{'Temperature': 0.019

In [40]:
print(decision_tree)

{'Outlook': {'Overcast': 'Yes', 'Rainy': {'Windy': {False: 'Yes', True: 'No'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
